In [ ]:
!pip install google-cloud-bigquery google-cloud-pubsub


In [ ]:
from google.cloud import bigquery

# Define your project, dataset, and table
project_id = "qwiklabs-gcp-02-3205ce60aeba"
dataset_id = "air_traffic_data"
table_id = "transponder_feed"

# Initialize BigQuery client
bq_client = bigquery.Client(project=project_id)

# Create dataset if it doesn't exist
dataset_ref = bigquery.Dataset(f"{project_id}.{dataset_id}")
bq_client.create_dataset(dataset_ref, exists_ok=True)

# Define schema
schema = [
    bigquery.SchemaField("MT", "STRING"),
    bigquery.SchemaField("TT", "INT64"),
    bigquery.SchemaField("SID", "STRING"),
    bigquery.SchemaField("AID", "STRING"),
    bigquery.SchemaField("Hex", "STRING"),
    bigquery.SchemaField("FID", "STRING"),
    bigquery.SchemaField("DMG", "DATE"),
    bigquery.SchemaField("TMG", "TIME"),
    bigquery.SchemaField("DML", "DATE"),
    bigquery.SchemaField("TML", "TIME"),
    bigquery.SchemaField("CS", "STRING"),
    bigquery.SchemaField("Alt", "INT64"),
    bigquery.SchemaField("GS", "INT64"),
    bigquery.SchemaField("Trk", "INT64"),
    bigquery.SchemaField("Lat", "FLOAT64"),
    bigquery.SchemaField("Lng", "FLOAT64"),
    bigquery.SchemaField("VR", "INT64"),
    bigquery.SchemaField("Sq", "STRING"),
    bigquery.SchemaField("Alrt", "INT64"),
    bigquery.SchemaField("Emer", "INT64"),
    bigquery.SchemaField("SPI", "INT64"),
    bigquery.SchemaField("Gnd", "INT64"),
]

# Create table
table_ref = bq_client.dataset(dataset_id).table(table_id)
table = bigquery.Table(table_ref, schema=schema)
bq_client.create_table(table, exists_ok=True)

print("BigQuery dataset and table created.")


BigQuery dataset and table created.


In [ ]:
from google.cloud import pubsub_v1
import csv
from datetime import datetime

subscriber = pubsub_v1.SubscriberClient()
subscription_path = subscriber.subscription_path("paul-leroy", "your-subscription-id")

def parse_and_insert(message):
    row = next(csv.reader([message.data.decode("utf-8")]))
    row += [''] * (22 - len(row))  # pad missing fields

    # Map to schema
    row_dict = dict(zip(
        [field.name for field in schema],
        row
    ))

    # Insert into BigQuery
    errors = bq_client.insert_rows_json(table_ref, [row_dict])
    if errors:
        print("Error inserting row:", errors)
    else:
        print("Row inserted.")
    message.ack()

streaming_pull_future = subscriber.subscribe(subscription_path, callback=parse_and_insert)
print("🚀 Listening for messages... (Ctrl+C to stop)")


🚀 Listening for messages... (Ctrl+C to stop)


In [ ]:
%%bigquery
SELECT COUNT(*)
FROM `qwiklabs-gcp-02-3205ce60aeba.air_traffic_data.transponder_feed`;


Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,0
